In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
from macd import compute_ema
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta
import plotly.graph_objs as go

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [4]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [5]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [6]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865


In [7]:
# infy_df.to_csv('dummy.csv')

In [8]:
df = infy_df.copy()

In [9]:
new_df = infy_df.copy()

# True range formula 

tr = true range = max[(high - low), abs(high - previous close), abs (low - previous close)]


# Steps to compute the Average Daily Index 

1.	Start off by calculating the +DM, -DM, and True Range (TR) for each period you are analyzing. Note:
1.	+DM = Current High - Previous High
2.	-DM = Previous Low - Current Low
2.	You can use +DM when the Current High - Previous High > Previous Low - Current Low.
3.	Use -DM when the Previous Low - Current Low > Current High - Previous High.
4.	The TR is the greater of the Current High - Current Low, the Current High - Previous Close, or the Current Low - Previous Close.
5.	Go ahead and smooth your period averages of +DM, -DM, and TR. Then, insert the -DM and +DM values to calculate the smoothed averages of those.
6.	First xTR = Sum of first x TR readings (x = number of…)
7.	Next xTR value = First xTR - (Prior xTR/14) + Current TR
8.	Then divide the smoothed +DM value by the smoothed TR value to get your +DI value. Multiply this value by 100.
9.	Divide the smoothed -DM value by the smoothed TR value to get your -DI value. Multiply this value by 100.
10.	The formula for the Directional Movement Index (DX) is +DI minus -DI, then divided by the sum of +DI and -DI (all of these are absolute values). Multiply this value by 100.
11.	In order to get the ADX, you’ll need to continue calculating the DX values for x periods. Smooth the results of the periods in order to get your ADX value.
12.	First ADX = the sum of x periods of DX / x
13.	Finally, ADX = ((Prior ADX * 13) + Current DX) / x

In [10]:
df['tr'] = np.maximum.reduce([df['high'] - df['low'],
                  abs(df['high'] - df['close'].shift(1)),
                  abs(df['low'] - df['close'].shift(1))])
# df

In [11]:
df.head()

,open,high,low,close,volume,tr
date,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,8.25
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,6.50
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,6.10
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,4.45


In [12]:
# +DM = Current High - Previous High
# # 2.	-DM = Previous Low - Current Low
# You can use +DM when the Current High - Previous High > Previous Low - Current Low.
# Use -DM when the Previous Low - Current Low > Current High - Previous High
# The TR is the greater of the Current High - Current Low, the Current High - Previous Close, or the Current Low - Previous Close.

df['+DM'] = np.where((df['high'] - df['high'].shift(1)) > (df['low'].shift(1) - df['low']),
                    np.maximum(df['high'] - df['high'].shift(1), 0),
                    0)

df['-DM'] = np.where((df['low'].shift(1) - df['low']) > (df['high'] - df['high'].shift(1)),
                    np.maximum(df['low'].shift(1) - df['low'], 0),
                    0)

df 

,open,high,low,close,volume,tr,+DM,-DM
date,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,NaN,0.0,0.00
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,8.25,0.0,4.05
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,6.50,0.0,2.70
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,6.10,0.0,2.70
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,4.45,0.0,2.45
...,...,...,...,...,...,...,...,...
2024-06-06 17:25:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00
2024-06-06 17:30:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00
2024-06-06 17:35:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00


In [13]:
df.to_csv('comp.csv')

In [14]:
for i in range(len(df)):
    if i == 14:
        df.loc[df.index[i-1], 'tr_smoothed'] = df['tr'][:i].sum()
        df.loc[df.index[i-1], '+DM_smoothed'] = df['+DM'][:i].sum()
        df.loc[df.index[i-1], '-DM_smoothed'] = df['-DM'][:i].sum()
        print(df.loc[df.index[i-1], 'tr_smoothed'])            

54.85000000000059


In [15]:
df.to_csv('comp.csv')

In [16]:
for i in range(14,len(df)):
    df.loc[df.index[i], 'tr_smoothed'] = (df.loc[df.index[i-1], 'tr_smoothed'] - (df.loc[df.index[i-1], 'tr_smoothed'] / 14)) + df.loc[df.index[i], 'tr'] 
    df.loc[df.index[i], '+DM_smoothed'] = (df.loc[df.index[i-1], '+DM_smoothed'] - (df.loc[df.index[i-1], '+DM_smoothed'] / 14)) + df.loc[df.index[i], '+DM'] 
    df.loc[df.index[i], '-DM_smoothed'] = (df.loc[df.index[i-1], '-DM_smoothed'] - (df.loc[df.index[i-1], '-DM_smoothed'] / 14)) + df.loc[df.index[i], '-DM'] 


In [17]:
df

,open,high,low,close,volume,tr,+DM,-DM,tr_smoothed,+DM_smoothed,-DM_smoothed
date,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,NaN,0.0,0.00,NaN,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,8.25,0.0,4.05,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,6.50,0.0,2.70,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,6.10,0.0,2.70,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,4.45,0.0,2.45,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06 17:25:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.634192,1.875369,0.814646
2024-06-06 17:30:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.088893,1.741414,0.756457
2024-06-06 17:35:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,6.582543,1.617028,0.702425


In [18]:
df['DI+'] = 100*(df['+DM_smoothed'] / df['tr_smoothed'])
df['DI-'] = 100*(df['-DM_smoothed'] / df['tr_smoothed'])
# df

In [19]:
df['DX'] = abs((df['DI+'] - df['DI-'])/(df['DI+'] + df['DI-']))*100
df

,open,high,low,close,volume,tr,+DM,-DM,tr_smoothed,+DM_smoothed,-DM_smoothed,DI+,DI-,DX
date,,,,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,8.25,0.0,4.05,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,6.50,0.0,2.70,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,6.10,0.0,2.70,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,4.45,0.0,2.45,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06 17:25:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.634192,1.875369,0.814646,24.565393,10.671022,39.431851
2024-06-06 17:30:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.088893,1.741414,0.756457,24.565393,10.671022,39.431851
2024-06-06 17:35:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,6.582543,1.617028,0.702425,24.565393,10.671022,39.431851


In [20]:
df.loc[df.index[14*2-1], 'ADX'] = df['DX'][14:14*2].mean()
for i in range(14*2, len(df)):
        df.loc[df.index[i], 'ADX'] = (
            (df.loc[df.index[i-1], 'ADX'] * (14 - 1) + df.loc[df.index[i], 'DX']) / 14
        )

df


,open,high,low,close,volume,tr,+DM,-DM,tr_smoothed,+DM_smoothed,-DM_smoothed,DI+,DI-,DX,ADX
date,,,,,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,8.25,0.0,4.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,6.50,0.0,2.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,6.10,0.0,2.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,4.45,0.0,2.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06 17:25:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.634192,1.875369,0.814646,24.565393,10.671022,39.431851,39.113905
2024-06-06 17:30:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.088893,1.741414,0.756457,24.565393,10.671022,39.431851,39.136615
2024-06-06 17:35:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,6.582543,1.617028,0.702425,24.565393,10.671022,39.431851,39.157704


In [21]:
df.to_csv('comp.csv')

In [22]:
def compute_adx(df,n):
    
    df['tr'] = np.maximum.reduce([df['high'] - df['low'],
                  abs(df['high'] - df['close'].shift(1)),
                  abs(df['low'] - df['close'].shift(1))])
    df['+DM'] = np.where((df['high'] - df['high'].shift(1)) > (df['low'].shift(1) - df['low']),
                    np.maximum(df['high'] - df['high'].shift(1), 0),
                    0)

    df['-DM'] = np.where((df['low'].shift(1) - df['low']) > (df['high'] - df['high'].shift(1)),
                        np.maximum(df['low'].shift(1) - df['low'], 0),
                        0)
    
    for i in range(len(df)):
        if i == n:
            df.loc[df.index[i-1], 'tr_smoothed'] = df['tr'][:i].sum()
            df.loc[df.index[i-1], '+DM_smoothed'] = df['+DM'][:i].sum()
            df.loc[df.index[i-1], '-DM_smoothed'] = df['-DM'][:i].sum()

    for i in range(n,len(df)):
        df.loc[df.index[i], 'tr_smoothed'] = (df.loc[df.index[i-1], 'tr_smoothed'] - (df.loc[df.index[i-1], 'tr_smoothed'] / n)) + df.loc[df.index[i], 'tr'] 
        df.loc[df.index[i], '+DM_smoothed'] = (df.loc[df.index[i-1], '+DM_smoothed'] - (df.loc[df.index[i-1], '+DM_smoothed'] / n)) + df.loc[df.index[i], '+DM'] 
        df.loc[df.index[i], '-DM_smoothed'] = (df.loc[df.index[i-1], '-DM_smoothed'] - (df.loc[df.index[i-1], '-DM_smoothed'] / n)) + df.loc[df.index[i], '-DM']

    df['DI+'] = 100*(df['+DM_smoothed'] / df['tr_smoothed'])
    df['DI-'] = 100*(df['-DM_smoothed'] / df['tr_smoothed']) 
    df['DX'] = (abs(df['DI+'] - df['DI-']) / (df['DI+'] + df['DI-'])) * 100
    df.loc[df.index[n*2-1], 'ADX'] = df['DX'][n:n*2].mean()

    for i in range(n*2, len(df)):
        df.loc[df.index[i], 'ADX'] = (
            (df.loc[df.index[i-1], 'ADX'] * (n - 1) + df.loc[df.index[i], 'DX']) / n
        )

    return df

In [23]:
new_df = compute_adx(new_df,14)
new_df

,open,high,low,close,volume,tr,+DM,-DM,tr_smoothed,+DM_smoothed,-DM_smoothed,DI+,DI-,DX,ADX
date,,,,,,,,,,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,8.25,0.0,4.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,6.50,0.0,2.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,6.10,0.0,2.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,4.45,0.0,2.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06 17:25:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.634192,1.875369,0.814646,24.565393,10.671022,39.431851,39.113905
2024-06-06 17:30:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,7.088893,1.741414,0.756457,24.565393,10.671022,39.431851,39.136615
2024-06-06 17:35:00+05:30,1472.25,1472.25,1472.25,1472.25,0,0.00,0.0,0.00,6.582543,1.617028,0.702425,24.565393,10.671022,39.431851,39.157704
